In [44]:
import pandas as pd
import json
from IPython.display import display

pd.set_option('display.max_rows', None) #pri ispisu se prikazuju svi redovi

Ucitavanje podataka i ispisivanje osnovnih informacija o njima.

In [45]:
with open('2. events.jsonl') as f:
    eventsJson = [json.loads(line) for line in f]

events = pd.json_normalize(eventsJson)

display(events.head())
events.info()

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os
0,28039,transaction,1.274573e+09,36835c60-5bcb-11ed-84c2-8afbeb371208,EUR,1.99,NaN,NaN,NaN
1,11528,login,1.274313e+09,1b53b16a-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
2,13381,login,1.274400e+09,17f29360-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
3,8075,login,1.273881e+09,184d5566-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
4,4340,registration,1.274462e+09,361d78d2-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,DE,Ing. Ekkehard Gude,Android


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28053 entries, 0 to 28052
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   event_id                         28053 non-null  int64  
 1   event_type                       28052 non-null  object 
 2   event_timestamp                  28051 non-null  float64
 3   event_data.user_id               28051 non-null  object 
 4   event_data.transaction_currency  634 non-null    object 
 5   event_data.transaction_amount    634 non-null    float64
 6   event_data.country               7492 non-null   object 
 7   event_data.name                  7492 non-null   object 
 8   event_data.device_os             7492 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 1.9+ MB


Od interesa su samo dogadjaji logovanja, registracije i transakcije. Pronadjimo sve dogadjaje koje nisu medju njima.

In [46]:
nonInterestingEvents = events[~events['event_type'].isin(['login', 'transaction', 'registration'])]
display(nonInterestingEvents)

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os
23309,2075116,concert,1.274572e+10,2cb2e110-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
26374,19283330,new_years_eve,1.274486e+09,2b16f152-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
26566,3618,NaN,NaN,30e3d17c-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,DE,Dariusz Hauffer,iOS


Dogadjaj koncerta i doceka Nove godine nam definitivno nisu od interesa u ovom zadatku (pogotovu sto se ova Nova godina docekuje 22. maja 2010. 1:53:20 GMT+02:00 DST :D). Medjutim interesantan je event_type NaN red. Da li mozda imamo dogadjaj sa istim id-jem koji nam vise govori o njemu?

In [47]:
events3618 = events[events['event_id'] == 3618]
display(events3618)

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os
26072,3618,registration,1.274460e+09,30e3d17c-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,DE,Dariusz Hauffer,iOS
26566,3618,NaN,NaN,30e3d17c-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,DE,Dariusz Hauffer,iOS


Dakle, ovaj red predstavlja neuspesno cuvanje dogadjaja registracije koga vec imamo. Prema tome sva tri dogadjaja mozemo izbaciti iz naseg skupa podataka.

In [48]:
events = events[events['event_type'].isin(['login', 'transaction', 'registration'])]
print(len(events))

28050


Sve kolone su odgovarajuceg tipa i non-null. Testirajmo sada obavezne kolone svakog dogadjaja na NaN vrednost.

In [49]:
display(events[events[['event_id', 'event_timestamp', 'event_data.user_id']].isna().any(axis=1)])

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os
26949,531414,registration,NaN,3d2c52e-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,US,Christine Tucker,Android
27376,1211145,login,1.271313e+09,NaN,NaN,NaN,NaN,NaN,NaN
27864,651913,registration,1.274530e+09,NaN,NaN,NaN,IT,Sig.ra Renata Jacuzzi,Android


Ovo su definitivno nevalidni event-ovi zato sto su navedena polja obavezna za svaki dogadjaj. Po uslovu zadatka ("Discard invalid events. Events are considered invalid if they do not meet the requirements specified above") sve nevalidne dogadjaje treba odbaciti. Ipak nije lose proveriti da li su ovo mozda duplirani podaci, kao i sve dogadjaje koje su vezane za njih.

Ako pogledamo prvi red videcemo da postoji user_id. Pretrazimo sve akcije ovog korisnika.

In [50]:
display(events[events['event_data.user_id'] == '3d2c52e-5bcb-11ed-84c2-8afbeb371208'])

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os
26949,531414,registration,NaN,3d2c52e-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,US,Christine Tucker,Android


Dakle, ovaj korisnik se ne spominje nigde u daljim dogadjajima. Zbog toga sto je ovo definitivno prljav podatak i akcije ovog korisnika ne postoje nigde drugde, mozemo mirne duse da uklonimo ovaj red u nastavku :D.

Za druga dva reda ne postoji korisnik. Pokusajmo da ih pronadjemo preko vremena i id dogadjaja.

In [51]:
display(events[(events['event_timestamp'] == 1.271313e+09) | (events['event_id'] == 1211145)])

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os
27376,1211145,login,1.271313e+09,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
display(events[(events['event_timestamp'] == 1.274530e+09) | (events['event_id'] == 651913)])

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os
27864,651913,registration,1.274530e+09,NaN,NaN,NaN,IT,Sig.ra Renata Jacuzzi,Android


Ni ova dva dogadjaja se ne javljaju nigde drugde i predstavljaju "izolovane" prljave podatke koje takodje treba da obrisemo.

In [53]:
events = events[~events[['event_id', 'event_timestamp', 'event_data.user_id']].isna().any(axis=1)]
print(len(events))

28047


Testirajmo sada da li su obavezna polja svakog dogadjaja prazni stringovi. Event_id i event_timestamp su numericki podaci, a kolonu event_type smo vec testirali. Ipak moze se desiti da je neciji user_id ostao prazan.

In [54]:
display(events[events['event_data.user_id'] == ''])

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os


Ne postoje korisnici sa praznim user_id-om.

Prebacimo kolonu event_timestamp u citljiviju formu vremena.

In [55]:
events['event_timestamp'] = pd.to_datetime(events['event_timestamp'], unit='s')

display(events.head())
events.info()

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os
0,28039,transaction,2010-05-22 23:57:47,36835c60-5bcb-11ed-84c2-8afbeb371208,EUR,1.99,NaN,NaN,NaN
1,11528,login,2010-05-19 23:52:11,1b53b16a-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
2,13381,login,2010-05-20 23:59:15,17f29360-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
3,8075,login,2010-05-14 23:53:29,184d5566-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
4,4340,registration,2010-05-21 17:15:43,361d78d2-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,DE,Ing. Ekkehard Gude,Android


<class 'pandas.core.frame.DataFrame'>
Int64Index: 28047 entries, 0 to 28052
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   event_id                         28047 non-null  int64         
 1   event_type                       28047 non-null  object        
 2   event_timestamp                  28047 non-null  datetime64[ns]
 3   event_data.user_id               28047 non-null  object        
 4   event_data.transaction_currency  634 non-null    object        
 5   event_data.transaction_amount    634 non-null    float64       
 6   event_data.country               7489 non-null   object        
 7   event_data.name                  7489 non-null   object        
 8   event_data.device_os             7489 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(6)
memory usage: 2.1+ MB


Da li mozda imamo neki dogadjaj van 8. maja 2010. - 22. maja 2010.?

In [56]:
outOfDateRange = events[(events['event_timestamp'] < '2010-05-08 00:00:00') | (events['event_timestamp'] > '2010-05-22 23:59:59')]
display(outOfDateRange)

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os
10707,137645,registration,1980-11-04 09:03:54,238271dd6-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,DE,Felix Heinrich-Girschner,Android
12808,21334285,login,1980-11-27 12:20:34,3b6b5656-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
21575,27971347,transaction,2010-04-29 20:24:38,34db3ac2-5bcb-11ed-84c2-8afbeb371208,USD,-1.99,NaN,NaN,NaN


Logovanje, registracija i transakcija pre pustanje igrice na produkciju?

Moguce da su u pitanju neki testni podaci koji su zaostali u bazi. Hajde da potrazimo sve akcije ovih korisnika, da bismo dobili dodatne informacije.

In [57]:
outOfDateUsersEvents = events[events['event_data.user_id'].isin(outOfDateRange['event_data.user_id'])]
display(outOfDateUsersEvents.sort_values(['event_data.user_id', 'event_timestamp']))

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os
10707,137645,registration,1980-11-04 09:03:54,238271dd6-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,DE,Felix Heinrich-Girschner,Android
21575,27971347,transaction,2010-04-29 20:24:38,34db3ac2-5bcb-11ed-84c2-8afbeb371208,USD,-1.99,NaN,NaN,NaN
13787,4165,registration,2010-05-21 18:41:19,34db3ac2-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,US,Matthew Esparza,Android
19586,27977,transaction,2010-05-22 23:57:58,34db3ac2-5bcb-11ed-84c2-8afbeb371208,USD,0.99,NaN,NaN,NaN
12808,21334285,login,1980-11-27 12:20:34,3b6b5656-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
8247,5073,registration,2010-05-22 15:53:00,3b6b5656-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,US,Lindsey Gibson,Android
11781,24285,login,2010-05-22 23:59:53,3b6b5656-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN


Prvi korisnik se registrovao 1980. godine (30 godina pre izbacivanja igre na produkciju :D) i nakon toga nije zabelezena nijedna akcija ovog korisnika. Mozda je u pitanju greska u belezenju samog vremena ili je to neki testni podatak. U svakom slucaju ovo je nevalidan podatak i po tekstu zadatka sve nevalidne podatke po bilo kom kriterijumu treba odbaciti.

Drugi korisnik je uradio transakciju pre same registracije. Da li je to dozvoljeno tekstom zadatka?
"On registration, the user is assigned a unique identifier which allows us to track him across all other events" - dakle registracija mora biti prva akcija korisnika -> transakcija 27977 je prljav (verovatno testni) podatak.

Treci korisnik je uradio logovanje 1980. godine + pre registracije -> prljav podatak.

Sva tri dogadjaja van dozvoljenog vremenskog opsega mozemo ukloniti jer ne ruse ostale akcije tih korisnika (npr. bilo bi otvoreno pitanje sta raditi ako bi registracija bila van dozvoljenog opsega vremena, a ostale akcije validne, ali takav slucaj nemamo :D).

In [58]:
events = events[(events['event_timestamp'] >= '2010-05-08 00:00:00') & (events['event_timestamp'] <= '2010-05-22 23:59:59')]
print(len(events))

28044


Predjimo sada na duplirane dogadjaje.

In [59]:
display(events[events.duplicated()])

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os
6198,11701,login,2010-05-19 23:55:08,1b91daf8-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
6655,27834,transaction,2010-05-22 01:39:03,2bafe02e-5bcb-11ed-84c2-8afbeb371208,USD,1.99,NaN,NaN,NaN
8543,20151,login,2010-05-21 23:59:46,17a3c00a-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
9093,1765,registration,2010-05-19 00:45:20,23871dd6-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,DE,Felix Heinrich-Girschner,Android
10120,7464,registration,2010-05-22 11:36:27,4d1f385e-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,DE,Elisabeth Hartmann,Android
10565,26936,login,2010-05-22 23:59:59,2057d3c6-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
10943,1513,registration,2010-05-19 03:17:57,21b5730e-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,DE,Muharrem Ortmann,iOS
27861,5312,registration,2010-05-22 13:21:16,3d241442-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,DE,Isabell Hahn-Schmiedecke,Web


Postoji nekoliko identicnih dupliranih dogadjaja. U slucaju realne aplikacije, ovo bi moglo da ukazuje na bag u cuvanju samih podataka o dogadjajima. 

Vidimo da ovaj problem nije vezan za konkretan tip dogadjaja, sistem uredjaja ili vremenski trenutak. Ipak, zanimljiva stvar da se to desavalo korisnicima DE pri registraciji. Cisto iz radoznalosti, provericu da li je ovo slucajnost.

In [60]:
registrationsDE = events[(events['event_type'] == 'registration') & (events['event_data.country'] == 'DE')][['event_id', 'event_timestamp', 'event_data.user_id', 'event_data.country', 'event_data.name', 'event_data.device_os']]
print(len(registrationsDE))

1893


Kao sto vidimo imamo mnogo korisnika iz Nemacke kojima se ovo nikad nije desilo pri registraciji, tako da je u pitanju cista slucajnost. Svakako, brisemo duplikate.

In [61]:
events = events.drop_duplicates()
print(len(events))

28036


Proverimo sada duplirane dogadjaje po svim ostalim kriterijumima osim event_id.

In [62]:
duplicatedEvents = events[events.duplicated(subset=events.columns.difference(['event_id']))]
print(len(duplicatedEvents))
display(duplicatedEvents.head(7))

207


,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os
1518,19653,login,2010-05-21 23:59:57,32ef59c8-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
2575,17037,login,2010-05-21 23:59:37,2b23e0d8-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
2739,26376,login,2010-05-22 23:59:57,4aa530b0-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
2807,25098,login,2010-05-22 23:59:58,2ff2002c-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
3061,25603,login,2010-05-22 23:59:53,29dc7762-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
3270,26856,login,2010-05-22 23:59:57,1e765b40-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
3522,18897,login,2010-05-21 23:59:57,38ca35d4-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN


Cak 207 dupliranih redova. Da li su sve duplirane akcije vezane za logovanje?

In [63]:
display(duplicatedEvents[duplicatedEvents['event_type'] != 'login'])

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os
27448,2715784,transaction,2010-05-21 23:55:24,1d67f3da-5bcb-11ed-84c2-8afbeb371208,EUR,1.99,NaN,NaN,NaN


Duplo logovanje od strane istog korisnika u istom vremenskom trenutku nije moguce i te duplirane redove treba odbaciti. Postavlja se pitanje sta raditi za dupliranu transakciju? Ako usvojimo pretpostavku da se transakcija belezi onda kada korisnik u aplikaciji kupi nesto, a ne kada novac legne na racun onda treba ukloniti i ovu dodatnu transakciju jer je nemoguce u istom trenutku izvrsiti dve transakcije.

In [64]:
events = events[~events['event_id'].isin(duplicatedEvents['event_id'])]
print(len(events))

27829


Da li su preostale jos neke akcije koje su se desile u istom trenutku od strane jednog korisnika.

In [65]:
duplicatedEvents = events[events.duplicated(['event_data.user_id', 'event_timestamp'], keep = False)]
display(duplicatedEvents.sort_values(['event_data.user_id', 'event_timestamp']))

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os
15379,27510,transaction,2010-05-18 23:17:48,1ea08dac-5bcb-11ed-84c2-8afbeb371208,EUR,9.900000e-01,NaN,NaN,NaN
18807,275110,transaction,2010-05-18 23:17:48,1ea08dac-5bcb-11ed-84c2-8afbeb371208,EUR,1.000000e+08,NaN,NaN,NaN
21903,2421113,registration,2010-05-20 05:25:57,28320012-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,Terraforming_Mars,Stacey King,iOS
26358,2413,registration,2010-05-20 05:25:57,28320012-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,US,Stacey King,iOS
8504,24752,login,2010-05-22 23:59:52,3c02420a-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
15764,5155,registration,2010-05-22 23:59:52,3c02420a-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,ES,Julia Prudencia Barbero Cazorla,iOS


Prvi slucaj: duplirane transakcije razlicitog iznosa. Vec smo imali duplirane transakcije i usvojili smo pretpostavku da se one ne smeju desiti u isto vreme. Ovde cemo odbaciti transakciju sa nevalidnom vrednoscu za kolonu event_data.transaction_amount.

Drugi slucaj: duplirana registracija. Svaki korisnik moze da se registruje samo jedanput (preciznije svaki registracioni id sme da se pojavi jedanput), pa cemo ovde odbaciti registraciju sa nevalidnom drzavom.

Treci slucaj: logovanje u istom trenutku kad i registracija. U nekim aplikacijama, odmah po registraciji korisnik biva automatski logovan u sistem. Ako se ta akcija belezi kao logovanje, onda bi i svi ostali korisnici imali ovakav scenario sto nije slucaj, tako da ovo jeste prljav podatak.

Pre brisanja treba potraziti ostale akcije ovog korisnika.

In [66]:
display(events[events['event_data.user_id'] == '3c02420a-5bcb-11ed-84c2-8afbeb371208'])

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os
8504,24752,login,2010-05-22 23:59:52,3c02420a-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
15764,5155,registration,2010-05-22 23:59:52,3c02420a-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,ES,Julia Prudencia Barbero Cazorla,iOS


Ovo su jedine dve akcije ovog korisnika. Ostaje otvoreno pitanje sta raditi sa ove dve akcije. Mozemo ih ukloniti obe, ili ukloniti samo logovanje. Po mom misljenju obe opcije su validne, ali ovde cemo pretpostaviti da se logovanje slucajno zadesilo prilikom registracije, tako da cemo ukloniti samo logovanje. Sada obrisimo sve ove prljave podatke.

In [67]:
events = events[~events['event_id'].isin(['275110', '2421113', '24752'])]
print(len(events))

27826


Posto registracija korisnika moze da se obavi tacno jedanput. Proverimo da li postoje igraci za koje su zabelezene visestruke registracije ili korisnici bez registracije, a koji su izvrsili transakciju i/ili logovanje.

In [68]:
registrationEventsByUser = events[events['event_type'] == 'registration'].groupby('event_data.user_id')
notOneRegistration = registrationEventsByUser.filter(lambda group: len(group) != 1)
display(notOneRegistration)

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os


In [69]:
#uniqueEvents = events['even']
noRegistrationEvents = events[events['event_type'] != 'registration']
noRegistrationEvents = noRegistrationEvents.drop_duplicates(subset=['event_data.user_id'])
print('Jedinstvenih korisnika koji su uradili logovanje i/ili transakciju: ', len(noRegistrationEvents))

registrationsEvent = events[events['event_type'] == 'registration']
print('Ukupno registracija: ', len(registrationsEvent))

numOfUniqueRegistrations = registrationsEvent[['event_data.user_id']].drop_duplicates().shape[0]
print('Ukupno jedinstvenih registracija: ', numOfUniqueRegistrations)

userWithoutReg = noRegistrationEvents[~noRegistrationEvents['event_data.user_id'].isin(registrationsEvent['event_data.user_id'])]
print('Broj korisnika bez registracije: ', len(userWithoutReg))

Jedinstvenih korisnika koji su uradili logovanje i/ili transakciju:  6146
Ukupno registracija:  7483
Ukupno jedinstvenih registracija:  7483
Broj korisnika bez registracije:  0


Dakle, svi korisnici trenutno imaju tacno jednu registraciju. Proverimo sada da li je ta registracija prva akcija svakog korisnika.

In [70]:
idx = events.groupby(['event_data.user_id'])['event_timestamp'].transform(min) == events['event_timestamp']

firstEventsByUser = events[idx]
display(firstEventsByUser[firstEventsByUser['event_type'] != 'registration'])

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os
15019,276317,transaction,2010-05-18 16:16:34,29620a54-5bcb-11ed-84c2-8afbeb371208,Potato,0.99,NaN,NaN,NaN


Dakle postoji korisnik cija je prva akcija bila transakcija, a ne registracija. Pretrazimo detaljnije akcije tog korisnika.

In [71]:
display(events[events['event_data.user_id'] == '29620a54-5bcb-11ed-84c2-8afbeb371208'].sort_values('event_timestamp'))

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os
15019,276317,transaction,2010-05-18 16:16:34,29620a54-5bcb-11ed-84c2-8afbeb371208,Potato,0.99,NaN,NaN,NaN
21993,2577,registration,2010-05-20 23:09:09,29620a54-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,DE,Ing. Hans-Henning Jockel B.A.,iOS
23411,13057,login,2010-05-20 23:30:34,29620a54-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
13758,27637,transaction,2010-05-20 23:49:54,29620a54-5bcb-11ed-84c2-8afbeb371208,EUR,0.99,NaN,NaN,NaN
20960,15196,login,2010-05-20 23:59:57,29620a54-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
17947,17563,login,2010-05-21 23:57:30,29620a54-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
5971,24017,login,2010-05-22 23:58:55,29620a54-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN
27348,24014,login,2010-05-22 23:59:46,29620a54-5bcb-11ed-84c2-8afbeb371208,NaN,NaN,NaN,NaN,NaN


Ostale akcije su validne. Zakljucak: samo krompir transakciju treba ukloniti :D

In [72]:
events = events[events['event_id'] != 276317]
len(events)

27825

Zanimljivo bi bilo sagledati da li je dozvoljeno izvrsiti transakciju pre logovanja na sistem. U tekstu zadatka pise da je transakcija in-app purchase tako da je logicno zakljuciti da je neophodno da korisnik bude ulogovan.

Potrazimo sve korisnike koji kao prvu ili kao prvih x akcija imaju transakciju (ne racunajuci registraciju). Pronadjimo za pocetak samo prve transakcije.

In [73]:
noRegEvents = events[events['event_type'] != 'registration'] # transakcije i logovanja

idsFirst = noRegEvents.groupby('event_data.user_id').event_timestamp.idxmin()
first = noRegEvents.loc[idsFirst]
print('Broj korisnika koji imaju transakciju za prvu akciju (ne racunajuci registraciju):', len(first[first['event_type'] == 'transaction']))

transactions = events[events['event_type'] == 'transaction']
print('Ukupan broj transakcija:', len(transactions))

Broj korisnika koji imaju transakciju za prvu akciju (ne racunajuci registraciju): 288
Ukupan broj transakcija: 629


Ovo je dosta veliki broj transakcija u odnosu na njihov ukupan broj. Usvojicemo pretpostavku da kada se korisnik uspesno registruje, on automatski biva ulogovan na sistem, ali se ta akcija ne smatra zasebnom akcijom logovanja. Prema tome, dozvoljeno je da se izvrsi transakcija i pre prvog zabelezenog logovanja.

Da li postoje dogadjaji sa podacima koji im ne pripadaju?

In [74]:
display(events[(events['event_type'] == 'login') & (events[['event_data.transaction_currency', 'event_data.transaction_amount', 'event_data.country', 'event_data.name', 'event_data.device_os']].notna().any(axis=1))])
display(events[(events['event_type'] == 'registration') & (events[['event_data.transaction_currency', 'event_data.transaction_amount']].notna().any(axis=1))])
display(events[(events['event_type'] == 'transaction') & (events[['event_data.country', 'event_data.name', 'event_data.device_os']].notna().any(axis=1))])

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os


,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os


,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os


Ne postoje dogadjaji koji imaju podatke koji im ne pripadaju.

Ispitajmo da li  su pri registraciji svi OS uredjaja u validnom skupu ("Valid values are iOS, Android and Web") i da li ima nevalidnih drzava.

In [75]:
registrationsEvents = events[events['event_type'] == 'registration']

validDeviceOS = ['iOS', 'Android', 'Web']
invalidDeviceOS = registrationsEvents[~registrationsEvents['event_data.device_os'].isin(validDeviceOS)]
display(invalidDeviceOS)


invalidCountry = registrationsEvents[registrationsEvents['event_data.country'].str.len() != 2]
display(invalidCountry)

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os


,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os
23662,713355,registration,2010-05-17 14:47:41,1c0afec-5bcb-11ed-842-8afbeb371208,NaN,NaN,Narnia,Kim Baker,Android


Sve registracije sada imaju validne OS uredjaja. Medjutim postoji transakcija koja ima nevalidnu zemlju (barem u realnom svetu :D) i zato je brisemo. 

In [76]:
events = events[events['event_id'] != 713355]
len(events)

27824

In [77]:
registrationsEvents = events[events['event_type'] == 'registration']

countriesCounter = registrationsEvents.groupby('event_data.country',sort=True).size()
display(countriesCounter)

event_data.country
DE    1889
ES    1884
IT    1825
US    1884
dtype: int64

Sada sve registracije imaju validnu drzavu (tj. njenu dvoslovnu oznaku). Sagledajmo ime korisnika pri registraciji.

In [78]:
display(registrationsEvents[registrationsEvents['event_data.name'].isna()])
display(registrationsEvents[registrationsEvents['event_data.name'] == ''])

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os


,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os


Iako nije obavezno polje pri registraciji, svi korisnici sada imaju ispravno ime.

Sagledajmo transakcije (tj. dozvoljenu valutu i visinu transakcije).

In [79]:
transactionsEvents = events[events['event_type'] == 'transaction']
validAmount = [0.99, 1.99, 2.99, 4.99, 9.99]

invalidAmount = transactionsEvents[~transactionsEvents['event_data.transaction_amount'].isin(validAmount)]
display(invalidAmount)

validCurrency = ['EUR', 'USD']
invalidCurrency = transactionsEvents[~transactionsEvents['event_data.transaction_currency'].isin(validCurrency)]
display(invalidCurrency)

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os


,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os
12396,279133166,transaction,2010-05-22 15:38:49,37e8952a-5bcb-11ed-84c2-8afbeb371208,Money,1.99,NaN,NaN,NaN


Sve transakcije sad imaju validan iznos uplate. Medjutim, transakcija koja koristi valutu "Money" nije validna i zato je uklanjamo.

In [80]:
events = events[events['event_id'] != 279133166]
len(events)

27823

Sagledajmo velicinu user_id.

In [81]:
display(events[events['event_data.user_id'].str.len() != 36])

,event_id,event_type,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,event_data.country,event_data.name,event_data.device_os


Svi dogadjaji imaju user_id iste velicine.

In [82]:
logins = events[events['event_type'] == 'login'][['event_id', 'event_timestamp', 'event_data.user_id']]
display(logins.head())
registrations = events[events['event_type'] == 'registration'][['event_id', 'event_timestamp', 'event_data.user_id', 'event_data.country', 'event_data.name', 'event_data.device_os']]
display(registrations.head())
transactions = events[events['event_type'] == 'transaction'][['event_id', 'event_timestamp', 'event_data.user_id', 'event_data.transaction_currency', 'event_data.transaction_amount']]
display(transactions.head())

,event_id,event_timestamp,event_data.user_id
1,11528,2010-05-19 23:52:11,1b53b16a-5bcb-11ed-84c2-8afbeb371208
2,13381,2010-05-20 23:59:15,17f29360-5bcb-11ed-84c2-8afbeb371208
3,8075,2010-05-14 23:53:29,184d5566-5bcb-11ed-84c2-8afbeb371208
5,8385,2010-05-15 23:59:02,18d5c252-5bcb-11ed-84c2-8afbeb371208
7,18432,2010-05-21 23:59:44,305ce5f4-5bcb-11ed-84c2-8afbeb371208


,event_id,event_timestamp,event_data.user_id,event_data.country,event_data.name,event_data.device_os
4,4340,2010-05-21 17:15:43,361d78d2-5bcb-11ed-84c2-8afbeb371208,DE,Ing. Ekkehard Gude,Android
6,341,2010-05-15 23:06:02,1946553a-5bcb-11ed-84c2-8afbeb371208,ES,Trinidad Plácido Río Soto,Web
17,4752,2010-05-21 01:25:26,3917f24c-5bcb-11ed-84c2-8afbeb371208,DE,Bianka Meyer,Web
20,4483,2010-05-21 09:17:40,37246254-5bcb-11ed-84c2-8afbeb371208,ES,Adán Ortiz Salas,iOS
24,2934,2010-05-20 12:27:42,2bf58930-5bcb-11ed-84c2-8afbeb371208,DE,Hans-Dieter Trupp,Android


,event_id,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount
0,28039,2010-05-22 23:57:47,36835c60-5bcb-11ed-84c2-8afbeb371208,EUR,1.99
152,27707,2010-05-21 23:54:19,1773880e-5bcb-11ed-84c2-8afbeb371208,USD,4.99
240,27498,2010-05-18 22:57:33,18bde29a-5bcb-11ed-84c2-8afbeb371208,USD,1.99
415,27608,2010-05-20 22:05:38,2568a1ec-5bcb-11ed-84c2-8afbeb371208,EUR,0.99
502,27619,2010-05-20 22:48:57,2b3a0020-5bcb-11ed-84c2-8afbeb371208,USD,0.99


In [83]:
with open('3. exchange_rates.jsonl') as f:
    exchangeRatesJson = [json.loads(line) for line in f]

exchangeRates = pd.json_normalize(exchangeRatesJson)

exchangeRates.info()
display(exchangeRates.head())

EUR_TO_USD = float(exchangeRates.iloc[1]['rate_to_usd'])
print('EUR_TO_USD: ', EUR_TO_USD)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   currency     2 non-null      object
 1   rate_to_usd  2 non-null      object
dtypes: object(2)
memory usage: 160.0+ bytes


,currency,rate_to_usd
0,USD,1
1,EUR,1.3


EUR_TO_USD:  1.3


In [84]:
import numpy as np
transactions['transaction_amount_usd'] = np.where(
        transactions['event_data.transaction_currency'] == 'EUR',
        EUR_TO_USD * transactions['event_data.transaction_amount'],
        transactions['event_data.transaction_amount']
    )
display(transactions.head())

,event_id,event_timestamp,event_data.user_id,event_data.transaction_currency,event_data.transaction_amount,transaction_amount_usd
0,28039,2010-05-22 23:57:47,36835c60-5bcb-11ed-84c2-8afbeb371208,EUR,1.99,2.587
152,27707,2010-05-21 23:54:19,1773880e-5bcb-11ed-84c2-8afbeb371208,USD,4.99,4.990
240,27498,2010-05-18 22:57:33,18bde29a-5bcb-11ed-84c2-8afbeb371208,USD,1.99,1.990
415,27608,2010-05-20 22:05:38,2568a1ec-5bcb-11ed-84c2-8afbeb371208,EUR,0.99,1.287
502,27619,2010-05-20 22:48:57,2b3a0020-5bcb-11ed-84c2-8afbeb371208,USD,0.99,0.990


Posto se dodatni poeni dobijaju ako se dalji koraci rade kroz SQL, izvezimo ove tabele u csv fajlove koje cemo importovati u SQL tabele.

In [85]:
logins.to_csv('logins.csv')
registrations.to_csv('registrations.csv')
transactions.to_csv('transactions.csv')